<h1> Beautiful Soup For The Soul </h1>
<hr>
<h2> Introduction </h2><hr>
So I heard you're looking for some data?  Well, you've come to the right place. 

But what do you do when the data you need doesn't come wrapped in a neat API?  Or that dataset doesn't have a "download as csv" button?  Inconcievable!  But never fear, web scraping is here!  

<br>
Web scraping is exactly what it sounds like: extracting data from websites.  But before we delve into Beautiful Soup, the web scraping library for Python, we have to take a step back and re-visit the good 'ole Myspace days editing HTML to make that perfect layout.  So strap in, and let's take a look at some basic HTML.

![](https://i.imgflip.com/r74yt.jpg)


<h2> HTML </h2><hr>
HTML is the standard markup language for creating web pages and web applications.  HTML is <b>not</b> a programming language, like Python — instead, it's a markup language that tells a browser how to display content. HTML is versy similar to programs like Microsoft Word where you can make text bold, create paragraphs, etc.

HTML consists of elements called tags. The most basic tag is the `<html>` tag. This tag tells the web browser that everything between the two tags can be expected to be HTML.

```html
<html>
    <head>
    </head>
    <body>
        <p class="bold-paragraph">
            Here's a paragraph of text!
            <a href="https://www.chase.com" id="banking">Bank with Chase!</a>
        </p>
        <p class="bold-paragraph extra-large">
            Here's a second paragraph of text!
            <a href="https://www.python.org" class="extra-large">Python</a>
        </p>
    </body>
</html>
```

<h3>Tag Genealogy:</h3>
    
<b>child</b> — a child is a tag inside another tag<br>
<b>parent</b> — a parent is the tag another tag is inside<br>
<b>sibling</b> — a sibiling is a tag that is nested inside the same parent as another tag

<h3>Tag Identifiers:</h3>

<b>class</b> —  One element can have multiple classes, and a class can be shared between elements<br>
<b>id</b> — Each element can only have one id, and an id can only be used once on a page

![](https://upload.wikimedia.org/wikipedia/commons/5/55/HTML_element_structure.svg)

<h2> Requests Library </h2><hr>
The first step in scraping a web page is retrieving the web page.  We can do this with the requests library.  The requests library will make a GET request to a web server, which will download the HTML contents of a given web page for us.

In [ ]:
import requests

page = requests.get("http://www.google.com")
page

After running our request, we get a `Response` object. This object has a status_code property, which indicates if the page was downloaded successfully



In [ ]:
page.status_code

A few possible responses:<br>
200 <b>OK</b> - The resource has been fetched and is transmitted in the message body.<br>
400 <b>Bad Request</b> - This response means that server could not understand the request due to invalid syntax.<br>
500 <b>Internal Server Error</b> The server has encountered a situation it doesn't know how to handle.<br>

We can print the HTML of the page:

In [ ]:
page.content

<h2> Beautiful Soup Library </h2><hr>
Now that we have the raw HTML, we can use what we can start parsing with our Soup!

In [ ]:
example_page = '''<!DOCTYPE html>
<html>
    <head>
    </head>
    <body>
        <p class="test1">Here's a paragraph of text!</p>
        <p class="test2">Here's a second paragraph of text!</p>
    </body>
</html>'''
#http://htmlpreview.github.io/?https://github.com/brianlau336/BeautifulSoupForTheSoul/blob/master/example_page1.html

In [ ]:
from bs4 import BeautifulSoup
soup = BeautifulSoup(example_page, 'html.parser')
print(soup.prettify())

In [ ]:
soup.children

In [ ]:
[x for x in soup.children] #list(soup.children)

In [ ]:
[type(item) for item in list(soup.children)]


In [ ]:
html = list(soup.children)[2]
html

In [ ]:
list(html.children)

In [ ]:
body = list(html.children)[3]
body

In [ ]:
p = list(body.children)[1]
p

In [ ]:
p.get_text()

<h3> Find All Tags </h3>
Okay, now that was somewhat painful.  Thankfully, we don't have to manually drill down like that every time.  If we want to extract a single tag, we can use the find_all method which will find all the instances of the tag on the page and return a list.

![](https://i.imgflip.com/2nn9l6.jpg)

In [ ]:
soup.find_all('p')


In [ ]:
for x in soup.find_all('p'):
    print (x.get_text().strip())

If you instead only want to find the first instance of a tag, you can use the find method, which will return a single `BeautifulSoup` object

In [ ]:
soup.find('p')

<h3> Searching By Genealogy/Relations </h3>
The Beautiful Soup API defines ten other methods for searching the tree, but don’t be afraid! Five of these methods are basically the same as find_all(), and the other five are basically the same as find(). The only differences are in what parts of the tree they search.



In [ ]:
first_p = soup.find('p')
first_p.find_parent()

In [ ]:
first_p = soup.find('p')
first_p.find_next_sibling()

<h3> Searching By Class/Id </h3>
Classes and ids are used by CSS to determine which HTML elements to apply certain styles to. We can also use them when scraping to specify specific elements we want to scrape. 



In [ ]:
example_page2 = '''<!DOCTYPE html>
<html>
    <head>
        <title>A simple example page</title>
    </head>
    <body>
        <div>
            <p class="inner-text first-item" id="first">
                First paragraph.
            </p>
            <p class="inner-text">
                Second paragraph.
            </p>
        </div>
        <p class="outer-text first-item" id="second">
        <b>
                First outer paragraph.
            </b>
        </p>
        <p class="outer-text">
            <b>
                Second outer paragraph.
            </b>
        </p>
    </body>
</html>'''
#http://htmlpreview.github.io/?https://github.com/brianlau336/BeautifulSoupForTheSoul/blob/master/example_page2.html
from bs4 import BeautifulSoup
soup = BeautifulSoup(example_page2, 'html.parser')
print(soup.prettify())

Now, we can use the `find_all` method to search for items by class or by id. In the below example, we'll search for any p tag that has the class outer-text:



In [ ]:
soup.find_all('p', class_='outer-text')


We can also search for elements by id:


In [ ]:
soup.find_all(id="first")


You can also search for items using CSS selectors. These selectors are how the CSS language allows developers to specify HTML tags to style. Here are some examples:

In [ ]:
soup.select("div p")

<h2> A Real Life Example - Houston Weather Data</h2><hr>
Let's say you wanted to grab some weather data for Houston, and you couldn't find any viable API's to use.  You stumble across the National Weather Service website and oh look! They have exactly what you want.  Time to flex those scraping skills.

![](https://i.pinimg.com/474x/b7/b9/17/b7b9175d8dba8065f71d0d7a8bba9d91--funny-weather-weather-memes.jpg)

Step 1: Download the web page containing the forecast.


In [87]:
import requests
from bs4 import BeautifulSoup
page = requests.get("https://forecast.weather.gov/MapClick.php?lat=29.7606&lon=-95.3697")
soup = BeautifulSoup(page.content, 'html.parser')

Step 2: Find the div with id "seven-day-forecast", and assign it to week_forecast

In [93]:
week_forecast = soup.find(id="seven-day-forecast")

Step 3: Inside week_forecast, find each individual forecast item 

In [94]:
forecast_items = week_forecast.find_all(class_="tombstone-container")

Step 4: Extract and print the first forecast item.

In [90]:
tonight = forecast_items[0]
print(tonight.prettify())

<div class="tombstone-container">
 <p class="period-name">
  Today
  <br/>
  <br/>
 </p>
 <p>
  <img alt="Today: Partly sunny, with a high near 70. South wind 5 to 10 mph. " class="forecast-icon" src="newimages/medium/bkn.png" title="Today: Partly sunny, with a high near 70. South wind 5 to 10 mph. "/>
 </p>
 <p class="short-desc">
  Partly Sunny
 </p>
 <p class="temp temp-high">
  High: 70 °F
 </p>
</div>


Okay cool, that forecast item (day) we drilled down into has everything we want!  
Looking at what we have, we can extract 4 major data points:

1. The name of the forecast item under "period-name" class<br>
2. The description of the conditions within the "title" property of img. This is a little different so we'll come back to it later.<br>
3. A short description of the conditions under "short-desc" class<br>
4. The temperature low under "temp" class<br>

In [95]:
period = tonight.find(class_="period-name").get_text()
short_desc = tonight.find(class_="short-desc").get_text()
temp = tonight.find(class_="temp").get_text()

print(period)
print(short_desc)
print(temp)

Today
Partly Sunny
High: 70 °F


Now, we can extract the "title" attribute from the img tag. To do this, we just treat the BeautifulSoup object like a dictionary, and pass in the attribute we want as a key:

In [96]:
img = tonight.find("img")
desc = img['title']

print(desc)

Today: Partly sunny, with a high near 70. South wind 5 to 10 mph. 


Now that we know how to extract each individual piece of information, we can combine our knowledge with css selectors and lists to extract everything at once.

So let's select all items with the class period-name inside an item with the class tombstone-container in seven_day.
We can use list comprehension to call the get_text method on each BeautifulSoup object.

In [97]:
period_tags = seven_day.select(".tombstone-container .period-name")
periods = [pt.get_text() for pt in period_tags]
periods

['Today',
 'Tonight',
 'Thursday',
 'ThursdayNight',
 'Friday',
 'FridayNight',
 'Saturday',
 'SaturdayNight',
 'Sunday']

In [98]:
short_descs = [sd.get_text() for sd in week_forecast.select(".tombstone-container .short-desc")]
temps = [t.get_text() for t in week_forecast.select(".tombstone-container .temp")]
descs = [d["title"] for d in week_forecast.select(".tombstone-container img")]

print(short_descs)
print(temps)
print(descs)

['Partly Sunny', 'Mostly Cloudy', 'ChanceShowers', 'ChanceShowers', 'ChanceT-storms', 'ChanceT-storms', 'ChanceT-storms thenSunny', 'Mostly Clear', 'Slight ChanceShowers']
['High: 70 °F', 'Low: 62 °F', 'High: 77 °F', 'Low: 66 °F', 'High: 78 °F', 'Low: 65 °F', 'High: 78 °F', 'Low: 57 °F', 'High: 75 °F']
['Today: Partly sunny, with a high near 70. South wind 5 to 10 mph. ', 'Tonight: Mostly cloudy, with a low around 62. South wind around 10 mph. ', 'Thursday: A 40 percent chance of showers after 7am.  Mostly cloudy, with a high near 77. South wind 5 to 10 mph. ', 'Thursday Night: A 40 percent chance of showers.  Mostly cloudy, with a low around 66. South wind around 10 mph. ', 'Friday: A chance of showers, with thunderstorms also possible after 7am.  Mostly cloudy, with a high near 78. Southeast wind around 10 mph.  Chance of precipitation is 40%.', 'Friday Night: A 40 percent chance of showers and thunderstorms.  Mostly cloudy, with a low around 65.', 'Saturday: A 30 percent chance of s

<h2>Pandas!</h2><hr>
Okay, so we have a bunch of lists with data...hm...does this look familiar? 

We can now combine the data into a Pandas DataFrame and analyze it.

In order to do this, we'll call the DataFrame class, and pass in each list of items that we have. We pass them in as part of a dictionary. Each dictionary key will become a column in the DataFrame, and each list will become the values in the column:

In [104]:
import pandas as pd
weather = pd.DataFrame({
        "period": periods, 
        "short_desc": short_descs, 
        "temp": temps, 
        "desc":descs
    })
weather

,period,short_desc,temp,desc
0,Today,Partly Sunny,High: 70 °F,"Today: Partly sunny, with a high near 70. Sout..."
1,Tonight,Mostly Cloudy,Low: 62 °F,"Tonight: Mostly cloudy, with a low around 62. ..."
2,Thursday,ChanceShowers,High: 77 °F,Thursday: A 40 percent chance of showers after...
3,ThursdayNight,ChanceShowers,Low: 66 °F,Thursday Night: A 40 percent chance of showers...
4,Friday,ChanceT-storms,High: 78 °F,"Friday: A chance of showers, with thunderstorm..."
5,FridayNight,ChanceT-storms,Low: 65 °F,Friday Night: A 40 percent chance of showers a...
6,Saturday,ChanceT-storms thenSunny,High: 78 °F,Saturday: A 30 percent chance of showers and t...
7,SaturdayNight,Mostly Clear,Low: 57 °F,"Saturday Night: Mostly clear, with a low aroun..."
8,Sunday,Slight ChanceShowers,High: 75 °F,Sunday: A 20 percent chance of showers after 7...


We can now do some analysis on the data. For example, we can use a regular expression and the Series.str.extract method to pull out the numeric temperature values:

In [109]:
temp_nums = weather["temp"].str.extract("(?P<temp_num>\d+)", expand=False)
weather["temp_num"] = temp_nums.astype('int')
temp_nums

0    70
1    62
2    77
3    66
4    78
5    65
6    78
7    57
8    75
Name: temp_num, dtype: object

We could then find the mean of all the high and low temperatures:



In [110]:
weather["temp_num"].mean()

69.77777777777777

We could also only select the rows that happen at night:


In [111]:
is_night = weather["temp"].str.contains("Low")
weather["is_night"] = is_night
is_night

0    False
1     True
2    False
3     True
4    False
5     True
6    False
7     True
8    False
Name: temp, dtype: bool

In [112]:
weather[is_night]


,period,short_desc,temp,desc,temp_num,is_night
1,Tonight,Mostly Cloudy,Low: 62 °F,"Tonight: Mostly cloudy, with a low around 62. ...",62,True
3,ThursdayNight,ChanceShowers,Low: 66 °F,Thursday Night: A 40 percent chance of showers...,66,True
5,FridayNight,ChanceT-storms,Low: 65 °F,Friday Night: A 40 percent chance of showers a...,65,True
7,SaturdayNight,Mostly Clear,Low: 57 °F,"Saturday Night: Mostly clear, with a low aroun...",57,True


<h2>Conclusion</h2><hr>

The reputation of web scraping has gotten a lot worse in the past few years, and for good reasons:

It's increasingly being used for business purposes to gain a competitive advantage. So there's often a financial motive behind it.
It's often done in complete disregard of copyright laws and of Terms of Service (ToS).
It's often done in abusive manners. For example, web scrapers might send much more requests per second than what a human would do, thus causing an unexpected load on websites. They might also choose to stay anonymous and not identify themselves. Finally, they might also perform prohibited operations on websites, like circumventing the security measures that are put in place to automatically download data, which would otherwise be inaccessible.
